# BERT 모델을 활용한 두 문장 관계 분류 학습

In [ ]:
!pip install transformers

In [ ]:
import torch
import sys

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

* 학습 데이터 확인
  * 두 문장과 label이 tab(\t)으로 구분되어있음

In [ ]:
data = open('/content/para_kqc_sim_data.txt', 'r', encoding='utf-8')
lines = data.readlines()

# 데이터셋 구조 확인
print(lines[0:10])

In [ ]:
import random
random.shuffle(lines)

* train data와 test data 구성
  * train data 80%, test data 20%

In [ ]:
train = {'sent_a':[], 'sent_b':[], 'label':[]}
test = {'sent_a':[], 'sent_b':[], 'label':[]}
for i, line in enumerate(lines):
    if i < len(lines) * 0.8:
        line = line.strip()
        train['sent_a'].append(line.split('\t')[0])
        train['sent_b'].append(line.split('\t')[1])
        train['label'].append(int(line.split('\t')[2]))
    else:
        line = line.strip()
        test['sent_a'].append(line.split('\t')[0])
        test['sent_b'].append(line.split('\t')[1])
        test['label'].append(int(line.split('\t')[2]))

In [ ]:
import pandas as pd

In [ ]:
train_data = pd.DataFrame({"sent_a":train['sent_a'], "sent_b":train['sent_b'], "label":train['label']})
test_data = pd.DataFrame({"sent_a":test['sent_a'], "sent_b":test['sent_b'], "label":test['label']})

* 중복 데이터 제거

In [ ]:
# 데이터 중복을 제외한 갯수 확인
print("학습데이터 : ",train_data.groupby(['sent_a', 'sent_b']).ngroups," 라밸 : ",train_data['label'].nunique())
# 학습데이터 :  15183  라밸 :  2
print("데스트 데이터 : ",test_data.groupby(['sent_a', 'sent_b']).ngroups," 라벨 : ",test_data['label'].nunique())
# 데스트 데이터 :  3796  라벨 :  2

# 중복 데이터 제거
train_data.drop_duplicates(subset=['sent_a', 'sent_b'], inplace= True)
test_data.drop_duplicates(subset=['sent_a', 'sent_b'], inplace= True)

# 데이터셋 갯수 확인
print('중복 제거 후 학습 데이터셋 : {}'.format(len(train_data)))
# 중복 제거 후 학습 데이터셋 : 15183
print('중복 제거 후 테스트 데이터셋 : {}'.format(len(test_data)))
# 중복 제거 후 테스트 데이터셋 : 3796

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

* null 데이터 제거

In [ ]:
# null 데이터 제거
train_data.replace('', np.nan, inplace=True)
test_data.replace('', np.nan, inplace=True)

train_data = train_data.dropna(how = 'any')
test_data = test_data.dropna(how = 'any')

print('null 제거 후 학습 데이터셋 : {}'.format(len(train_data)))
# null 제거 후 학습 데이터셋 : 15183
print('null 제거 후 테스트 데이터셋 : {}'.format(len(test_data)))
# null 제거 후 테스트 데이터셋 : 3796

In [ ]:
print(train_data['sent_a'][0])
# 오늘 관악구 습도는?
print(train_data['sent_b'][0])
# 오늘 관악구 습도 알고싶습니다.
print(train_data['label'][0])
# 1

In [ ]:
# 학습 전제 문장 길이조사
print('학습 전제 문장의 최대 길이 :',max(len(l) for l in train_data['sent_a']))
# 학습 전제 문장의 최대 길이 : 49
print('전제 문장의 평균 길이 :',sum(map(len, train_data['sent_a']))/len(train_data['sent_a']))
# 전제 문장의 평균 길이 : 22.360995850622405

plt.hist([len(s) for s in train_data['sent_a']], bins=50)
plt.xlabel('length of data')
plt.ylabel('number of data')
plt.show()

# 학습 가정 문장 길이조사
print('학습 가정 문장의 최대 길이 :',max(len(l) for l in train_data['sent_b']))
# 학습 가정 문장의 최대 길이 : 65
print('가정 문장의 평균 길이 :',sum(map(len, train_data['sent_b']))/len(train_data['sent_b']))
# 가정 문장의 평균 길이 : 25.547322663505238

plt.hist([len(s) for s in train_data['sent_b']], bins=50)
plt.xlabel('length of data')
plt.ylabel('number of data')
plt.show()

* BERT를 사용하여 학습

In [ ]:
# Store the model we want to use
from transformers import AutoModel, AutoTokenizer, BertTokenizer
MODEL_NAME = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

* tokenizer에서 두 문장 관계 분류 task에서 문장 2개를 input으로 넣음
  * tokenizer가 자동으로 `[CLS] sentenceA [SEP] sentenceB [SEP]` 형태로 token을 부착하여 tokenizing을 함
  * token_type_ids를 segmentA는 0, segmentB는 1로 tagging함

* train data 전체를 한번에 embedding함
  * input : list => output : list

In [ ]:
tokenized_train_sentences = tokenizer(
    list(train_data['sent_a'][0:]),
    list(train_data['sent_b'][0:]),
    return_tensors="pt",
    padding=True,
    truncation=True,
    add_special_tokens=True,
    max_length=64
    )

* `attention_mask` : 실제 분류를 위해 사용되는 데이터는 1, 나머지는 0으로 tagging됨

In [ ]:
print(tokenized_train_sentences[0])
# Encoding(num_tokens=64, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
print(tokenized_train_sentences[0].tokens)
# ['[CLS]', '오', '##늘', '관', '##악', '##구', '습', '##도는', '?', '[SEP]', '오', '##늘', '관', '##악', '##구', '습', '##도', '알', '##고', '##싶', '##습', '##니다', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
print(tokenized_train_sentences[0].ids)
# [101, 9580, 118762, 8900, 119110, 17196, 9482, 60884, 136, 102, 9580, 118762, 8900, 119110, 17196, 9482, 12092, 9524, 11664, 119088, 119081, 48345, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
print(tokenized_train_sentences[0].attention_mask)
# [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


 * 평가를 위한 test data tokenizing

In [ ]:
tokenized_test_sentences = tokenizer(
    list(test_data['sent_a'][0:]),
    list(test_data['sent_b'][0:]),
    return_tensors="pt",
    padding=True,
    truncation=True,
    add_special_tokens=True,
    max_length=64
    )

* label 저장 및 확인

In [ ]:
train_label = train_data['label'].values[0:]
test_label = test_data['label'].values[0:]

In [ ]:
print(train_label[0]) # 1

* `__getitem__()` : step이 진행됨에 따라 model에 지속적으로 입력되는 데이터
  * input : tokenizer를 통해서 나온 결과(key, value)와 사전에 정의된 label

In [ ]:
class MultiSentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [ ]:
train_dataset = MultiSentDataset(tokenized_train_sentences, train_label)
test_dataset = MultiSentDataset(tokenized_test_sentences, test_label)

* BERT를 활용하여 train
  * model 입장에서 한 문장이든지 두 문장이든지 상관없이 tokenize된 sentence가 input으로 들어가고, 마지막에 [CLS] token 하나만 분류를 하기 때문에 단일문장분류(`BertForSequenceClassification`)에서 사용했던 model 사용이 가능함

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments,  BertConfig

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    logging_dir='./logs',            # directory for storing logs
    logging_steps=500,
    save_total_limit=2,
)

* model initialized

In [ ]:
model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2) 
model.parameters
model.to(device)

* evaluation 결과 출력하는 `compute_metrics` 함수 구현

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,             # evaluation dataset
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate(eval_dataset=test_dataset)
'''
{'epoch': 3.0,
 'eval_accuracy': 0.9802423603793466,
 'eval_f1': 0.9792186201163757,
 'eval_loss': 0.0979667603969574,
 'eval_precision': 0.9746276889134032,
 'eval_recall': 0.9838530066815144,
 'eval_runtime': 7.0701,
 'eval_samples_per_second': 536.911}
'''

In [ ]:
trainer.save_model('./results')

In [ ]:
# native training using torch

# bert_config = BertConfig.from_pretrained(MODEL_NAME)
# bert_config.num_labels = 3
# model = BertForSequenceClassification(bert_config) 
# model.to(device)
# model.train()

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# optim = AdamW(model.parameters(), lr=5e-5)

# for epoch in range(3):
#     for batch in train_loader:
#         optim.zero_grad()
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)
#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs[0]
#         loss.backward()
#         optim.step()

* prediction 함수

In [ ]:
# predict함수
# 0: "non_similar", 1: "similar"
def sentences_predict(sent_A, sent_B):
    model.eval()
    tokenized_sent = tokenizer(
            sent_A,
            sent_B,
            return_tensors="pt",
            truncation=True,
            add_special_tokens=True,
            max_length=64
    )
    
    tokenized_sent.to('cuda:0')
    with torch.no_grad():# 그라디엔트 계산 비활성화
        outputs = model(
            input_ids=tokenized_sent['input_ids'],
            attention_mask=tokenized_sent['attention_mask'],
            token_type_ids=tokenized_sent['token_type_ids']
            )

    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    result = np.argmax(logits) # softmax를 통과하고 나온 가장 높은 점수를 가진 index를 return

    if result == 0:
      result = 'non_similar'
    elif result == 1:
      result = 'similar'
    return result

In [ ]:
print(sentences_predict('오늘 날씨가 어때요?','오늘의 날씨를 알려줘')) # similar
print(sentences_predict('오늘 날씨가 어때요?','기분 진짜 안좋다.')) # non_similar
print(sentences_predict('오늘 날씨가 어때요?','오늘 기분 어떠세요?')) # non_similar
print(sentences_predict('오늘 날씨가 어때요?','오늘 기분이 어때요?')) # non_similar
print(sentences_predict('오늘 날씨가 어때요?','지금 날씨가 어때요?')) # non_similar
print(sentences_predict('무협 소설 추천해주세요.','무협 장르의 소설 추천 부탁드립니다.')) # similar
print(sentences_predict('무협 소설 추천해주세요.','판타지 소설 추천해주세요.')) # non_similar
print(sentences_predict('무협 소설 추천해주세요.','무협 느낌나는 소설 하나 추천해주실 수 있으실까요?')) # similar